In [3]:
import concurrent.futures
import csv
import json
import logging
import os
import random
import sys
from heapq import nlargest

import openai
import torch
from tenacity import retry, stop_after_attempt, wait_exponential
from transformers import AutoModel, AutoTokenizer

TOP_K = 10
PART_NUM = 0
starting = 422328
error_list=[]
i = 424
openai.api_key = "31b8638c9eea48709a596501490f9e88"
openai.api_base = "https://gersteinbiocodeeval-eastus2.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

key_bundles = [
    ('aaccba8e27374383beb397ecdc615ee5', "https://biocodeeval-openai.openai.azure.com/"),
    ('31b8638c9eea48709a596501490f9e88', "https://gersteinbiocodeeval-eastus2.openai.azure.com/"),
    ('aaccba8e27374383beb397ecdc615ee5', "https://biocodeeval-openai.openai.azure.com/")
]

# Contriever part
tokenizer = AutoTokenizer.from_pretrained('facebook/contriever')
model = AutoModel.from_pretrained('facebook/contriever')

# Mean pooling
def mean_pooling(token_embeddings, mask):
    token_embeddings = token_embeddings.masked_fill(~mask[..., None].bool(), 0.)
    sentence_embeddings = token_embeddings.sum(dim=1) / mask.sum(dim=1)[..., None]
    return sentence_embeddings

# prompt_construction for questions
def construct_prompt(question, evidences):
    prompt_str = "Please mimic the science paper abstract and coresponding title below:\n{evidences}\n Please generate a title for this paper, Abstract:\n{question}.\nTitle:"
    evidences_str = "\n".join(evidences)
    prompt_str = prompt_str.replace("{evidences}",evidences_str)
    prompt_str = prompt_str.replace("{question}",question)
    return prompt_str

def construct_prompt_for_history(evidence):
    prompt_str = "Given the abstract of a science paper: "+evidence["Abstract"]+"\n and here's the correponding title. "+evidence["Title"]+"\nPlease summary the tone, text lenght and style features, which could be helpful in title generation, each in a new line:"
    return prompt_str

with open('task_5_test_questions.json', 'r', encoding='utf-8') as f:
    jsonObject = json.load(f)
f.close()


profile_history = {}
input_dict = {}
profile_dict = {}
for item in jsonObject:
    input_dict[item["id"]] = item["input"]
    input_str = input_dict[item["id"]].replace("Generate a title for the following abstract of a paper: ","")
    profile_dict[item["id"]] = [input_str]
    profile_history[item["id"]] = [{"Abstract":"","Title":input_str}]
    for profile in item["profile"]:
        profile_history[item["id"]].append({"id":profile["id"],"Abstract":profile["abstract"],"Title":profile["title"]})
        profile_dict[item["id"]].append("Abstract: "+profile["abstract"]+"\nTitle: "+profile["title"])

with open("cot_prompt_input_history_10_examples_test_2.tsv", "wb", buffering=0) as out_file:
    with torch.no_grad():
        for indice in profile_dict:
            if int(indice) >= starting:
                # Apply tokenizer
                inputs = tokenizer(profile_dict[indice], padding=True, truncation=True, return_tensors='pt')

                # Compute token embeddings
                outputs = model(**inputs)
                embeddings = mean_pooling(outputs[0], inputs['attention_mask'])

                # recall TOP_K evidences
                score_index = []
                for i in range(1,len(profile_dict[indice])):
                    score_index.append({"ordinal":i,"score":float((embeddings[0] @ embeddings[i]).cpu().detach())})
                top_score_index = nlargest(TOP_K, score_index, key=lambda item: item["score"])
                top_score_docs = [profile_history[indice][i] for i in (i["ordinal"] for i in top_score_index)]

                # build prompt with history and question 
                for item in top_score_docs:
                    prompt = construct_prompt_for_history(item)
                    my_dict = {}
                    my_dict["role"] = "user"
                    my_dict["content"] = prompt
                    l = []
                    l.append(my_dict)
                    try:
                        key_bundle = key_bundles[i%3]
                        openai.api_key, openai.api_base = key_bundle
                        response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                        result = response.choices[0].message["content"].replace('\n', ' ')
                        write_str = bytes(indice+"\t"+item["id"]+"\t"+result+"\n", 'utf-8')
                        out_file.write(write_str)
                        print(indice+"\t"+item["id"]+"\t"+result+"\n")
                        i = i + 1
                    except openai.error.Timeout:
                        print("Timeout",indice)
                        key_bundle = key_bundles[i%3]
                        openai.api_key, openai.api_base = key_bundle
                        try:
                            response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                            result = response.choices[0].message["content"].replace('\n', ' ')
                            write_str = bytes(indice+"\t"+item["id"]+"\t"+result+"\n", 'utf-8')
                            out_file.write(write_str)
                            print(indice+"\t"+item["id"]+"\t"+result+"\n")
                            i = i + 1
                        except openai.error.Timeout:
                            print("Timeout",indice)
                            key_bundle = key_bundles[i%3]
                            openai.api_key, openai.api_base = key_bundle
                            try:
                                response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                                result = response.choices[0].message["content"].replace('\n', ' ')
                                write_str = bytes(indice+"\t"+item["id"]+"\t"+result+"\n", 'utf-8')
                                print(indice+"\t"+item["id"]+"\t"+result+"\n")
                                out_file.write(write_str)
                            except Exception as e:
                                print(f"An error occurred: {str(e)}")
                                error_list.append(i)
                                continue
                        except Exception as e:
                            print(f"An error occurred: {str(e)}")
                            error_list.append(i)
                        #print(result)
                    except openai.error.InvalidRequestError:
                        print("InvalidRequestError",indice)
                        error_list.append(i)
                        #print(result)
                    except Exception as e:
                        print(f"An error occurred: {str(e)}")
                        error_list.append(i)
                    i = i + 1

422328	42232863	Tone: informative, technical Text length: medium (one paragraph) Style features: formal language, use of technical terms, clear and concise writing, objective tone, logical structure

422328	42232844	Tone: informative, objective Text length: moderate Style features: technical language, specific terminology, clear and concise writing, use of acronyms (HTTP)

422328	42232852	Tone: Informative, technical Text length: Medium Style features: Formal, descriptive, specific

422328	42232811	Tone: informative, technical Text length: concise, 6 words Style features: use of abbreviations (WWW, SGI), specific technology focus, emphasis on mobility and health management

422328	42232839	Tone: informative Text length: concise (5 words) Style features: use of alliteration, use of technical terms (smart card, cookies management)

422328	42232826	Tone: Neutral, informative Text length: 202 words Style features:  - Formal language - Technical terminology - Use of abbreviations (XML, VRML